<a href="https://colab.research.google.com/github/kbandesz/QGenAI/blob/main/QGenAI_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bloom classifier for learning objectives (LangChain implementation)

In [ ]:
# Set up environment XXX
import os
from google.colab import userdata

# Set Google Drive paths
working_dir = "/content/drive/MyDrive/Colab/QGenAI"
os.chdir(working_dir)

print(f"Current working directory set to: {os.getcwd()}")

# Set OpenAI API key
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
print(f"OpenAI API key set.")

Current working directory set to: /content/drive/MyDrive/Colab/QGenAI
OpenAI API key set.


In [ ]:
# Install packages
!pip install langchain langchain-openai langchain-community docx2txt

In [ ]:
# Import LangChain packages
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.document_loaders import Docx2txtLoader

In [ ]:
# Initialize the Language Model (LLM)
model = "gpt-4o-mini" #"gpt-4.1"
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model=model,
    temperature=0.0,    # For deterministic output
    max_tokens=1024
)

In [ ]:
# Define the Prompt Template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert in instructional design and educational pedagogy, specializing in Bloom's Taxonomy and curriculum analysis."),
    ("human", """Your task is to determine the Bloom’s Taxonomy level of a given learning objective based on the provided course materials.

COURSE MATERIALS:
--------------------------------
{course_material}
--------------------------------

LEARNING OBJECTIVE:
--------------------------------
"{learning_objective}"
--------------------------------

BLOOM’s TAXONOMY WITH EXAMPLES:
--------------------------------
Critically analyze the learning objective and course materials.
Remember, the Bloom's level is determined not just by the action verb in the learning objective, but by the highest level of cognitive complexity required to achieve the objective *within the specific context and scope of the provided course materials*.
Sample learning objectives pertain to a macroeconomic statistics course on the compilation of Producer Price Indexes (PPIs).

## Bloom’s Level 1: Remember
# Explanation: Learners recall facts, terms, definitions, and concepts. The focus is on memorization and pure recall without deeper interpretation of information.
# Sample Learning Objective: Identify the components of a basic price for an output PPI.

## Bloom’s Level 2: Understand
# Explanation: Learners demonstrate the ability to explain or interpret information in their own words. The focus is on understanding ideas or concepts.
# Sample Learning Objective: Distinguish between market and nonmarket producers for a PPI.

## Bloom’s Level 3: Apply
# Explanation: Learners apply what they have learned to diverse contexts outside the course. The focus is on using information in new situations.
# Sample Learning Objective: Identify whether an entity is a market or a nonmarket producer.

## Bloom’s Level 4: Analyze
# Explanation: Learners break down information into its constituent parts and examine the connections among them. The focus is on drawing far and wide connections (for example, recognizing patterns, determining cause and effect, comparing and contrasting ideas, etc.).
# Sample Learning Objective: Examine the challenges involved in creating PPIs for services activities in your country.

## Bloom’s Level 5: Evaluate
# Explanation: Learners critique information based on well-reasoned judgements (for example, quality, relevance, etc.). The focus is on assessing information using diverse perspectives, forming opinions, and being able to support or defend these opinions.
# Sample Learning Objective: Critically assess the PPI compilation process in your country for a chosen sector.

## Bloom’s Level 6: Create
# Explanation: Learners produce new or original work. The focus is on synthesizing information, imagining creative possibilities, and expressing originality in the output.
# Sample Learning Objective: Devise a statistical survey for collecting data from manufacturing businesses in your country to calculate an appropriate PPI.
--------------------------------

Based on the course materials and the learning objective, what is the Bloom level of the learning objective? Consider the cognitive demands implied by the learning objective in relation to the content and tasks suggested by the course materials. Your entire response must be a single, valid JSON object. Do not include any text or explanations outside of this JSON object. The JSON object should have the following keys:
- "bloom_level_1": The primary Bloom's level determined for the learning objective. Must be one of the exact strings: 'Remember', 'Understand', 'Apply', 'Analyze', 'Evaluate', or 'Create'.
- "rationale": A concise explanation (2-3 sentences) detailing *why* the learning objective, in conjunction with the provided course materials, maps to the chosen `bloom_level_1`. Refer to specific aspects of the definition of the Bloom level and how they apply in the context of the course material. Do NOT repeat the learning objective in the rationale.
- "ambiguous": A string indicating if there's a strong case for a second Bloom's level. Must be one of the exact strings: 'Yes' or 'No'.
- "ambiguous_reason": If 'ambiguous' is 'Yes', provide a concise explanation (2-3 sentences) for why the learning objective could also be classified at `bloom_level_2`. If 'ambiguous' is 'No', this field should be an empty string "".
- "bloom_level_2": If 'ambiguous' is 'Yes', provide the alternative Bloom's level. Must be one of the exact strings: 'Remember', 'Understand', 'Apply', 'Analyze', 'Evaluate', or 'Create'. If 'ambiguous' is 'No', this field should be an empty string "".
""")
])


In [ ]:
# Define the Output Parser
json_parser = JsonOutputParser()

# Create the LangChain (LCEL Chain)
bloom_chain = prompt_template | llm | json_parser


In [ ]:
# Load course material using Docx2txtLoader
from google.colab import files
import os

def load_course_material():
    print("Please upload the course material file (.docx):")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded. Exiting.")
        quit()

    # Get the name of the uploaded file (assuming only one file is uploaded)
    file_name = list(uploaded.keys())[0]
    file_content = uploaded[file_name]

    # Define a temporary file path
    temp_file_path = "uploaded_course_material.docx"

    # Save the uploaded content to the temporary file
    with open(temp_file_path, 'wb') as f:
        f.write(file_content)

    loader = Docx2txtLoader(temp_file_path)
    documents = loader.load()
    course_material = documents[0].page_content

    # Delete the temporary file
    os.remove(temp_file_path)

    # Basic check for empty content (minimal error handling for prototype)
    if not course_material.strip():
        print("Warning: The loaded course material is empty.")
        quit()

    return course_material

In [ ]:
# Read learning objectives from a file
from google.colab import files

def read_learning_objectives():
    print("Please upload the learning objectives file (.txt):")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded. Exiting.")
        quit()

    # Get the name of the uploaded file (assuming only one file is uploaded)
    file_name = list(uploaded.keys())[0]
    file_content_bytes = uploaded[file_name]

    # Decode bytes to string
    file_content_string = file_content_bytes.decode('utf-8')

    # Process the string content
    learning_objectives = [] # Initialize an empty list
    lines = file_content_string.splitlines()
    learning_objectives = [line.strip() for line in lines if line.strip()]

    return learning_objectives

In [ ]:
# Get LLM response
def invoke_bloom_chain(course_material, learning_objective):
    result = bloom_chain.invoke({
        "course_material": course_material,
        "learning_objective": learning_objective
    })

    return result

In [ ]:
# Invoke the chain for each learning objective in each module
print("-------------------------------------------")
print("--- Macroeconomic Policy Communications ---")
print("-------------------------------------------")

for m in range(2):
    course_material = load_course_material() # Removed file path argument
    learning_objectives = read_learning_objectives() # Removed file path argument
    n = len(learning_objectives)
    print(f"\n*** MODULE {m+1}: Analyzing {n} learning objectives ***")

    for idx, current_learning_objective in enumerate(learning_objectives):
        print(f"\nLearning Objective {idx + 1}/{n}: \"{current_learning_objective}\"\n")
        result = invoke_bloom_chain(course_material, current_learning_objective)

        # --- Display Result  ---
        print(f"Determined Bloom's Level (Primary): {result['bloom_level_1']}\n")
        print(f"Rationale: {result['rationale']}")
        print(f"Is Classification Ambiguous?: {result['ambiguous']}\n")

        # Check the 'ambiguous' flag and display related info if 'Yes'
        if result['ambiguous'] == 'Yes':
            print(f"Reason for Ambiguity: {result['ambiguous_reason']}\n")
            print(f"Alternative Bloom's Level: {result['bloom_level_2']}\n")
        print("--------------------------------------")


-------------------------------------------
--- Macroeconomic Policy Communications ---
-------------------------------------------

*** MODULE 1: Analyzing 4 learning objectives ***

Learning Objective 1/4: "Summarize the historical context of institutional policy communication."

Determined Bloom's Level (Primary): Understand

Rationale: The learning objective requires learners to explain the historical context of institutional policy communication, which involves interpreting and conveying information in their own words. This aligns with the 'Understand' level of Bloom's Taxonomy, as it goes beyond mere recall and requires comprehension of the concepts presented in the course materials.
Is Classification Ambiguous?: No

--------------------------------------

Learning Objective 2/4: "Identify the main benefits/purposes of transparency."

Determined Bloom's Level (Primary): Understand

Rationale: The learning objective requires learners to identify the main benefits and purposes of t